# Conversion of Regular Expression to NFA

In [27]:
from graphviz import Digraph
import uuid
epsilon = '\u03B5'

# Class to represent a NFA

The class NFA and its properties are defined. Operator overloading is used to Process sub-NFA's

In [28]:
class NFA:
    def __init__(self,trans='#'):
        self.start = uuid.uuid1().hex
        self.final = uuid.uuid1().hex
        self.states=set([self.start,self.final])
        self.transition=set()
        if trans != '#':
            self.transition.add((self.start,self.final,trans))
        
    def __or__(self, other):
        temp = NFA()
        temp.states.update(self.states,other.states)
        temp.transition.update(self.transition,other.transition)
        new = set([(temp.start,self.start,epsilon),(temp.start,other.start,epsilon),(self.final,temp.final,epsilon),(other.final,temp.final,epsilon)])
        temp.transition.update(new)
        return temp
    def __invert__(self):
        temp = NFA()
        temp.states.update(self.states)
        temp.transition.update(self.transition)
        new = set([(temp.start,temp.final,epsilon),(temp.start,self.start,epsilon),(self.final,temp.final,epsilon),(self.final,self.start,epsilon)])
        temp.transition.update(new)
        return temp
    def __and__(self, other):
        temp = NFA()
        temp.states.update(self.states,other.states)
        temp.transition.update(self.transition,other.transition)
        new = set([(temp.start,self.start,epsilon),(self.final,other.start,epsilon),(other.final,temp.final,epsilon)])
        temp.transition.update(new)
        return temp

# Conversion of Regular Expression to Postfix expression

In [29]:
def RegexToPostfix(regex):
    regex = '(' + regex + ')'
    postfix = ''
    optor = []
    opand = []
    i = 0
    while i < len(regex):
        c = regex[i]
        if c == '(':
            optor.append(c)
        elif c == '+': 
            optor.append('|')
        elif c == '.': 
            optor.append('&')
        elif c == ')':
            k = optor.pop()
            while k != '(':
                opand.append(k)
                k = optor.pop()
            if i+1 < len(regex) and regex[i+1] == '*':
                opand.append('~')
                i += 1
        else: 
            opand.append(c)
            if i+1 < len(regex) and regex[i+1] == '*':
                opand.append('~')
                i+=1

        i+=1
    postfix = postfix.join(opand)
    return postfix
        

# Using stack to obtain epsilon NFA

In [30]:
def ConvertToNFA(postfix):
    nfastack = []
    for c in postfix:
        if c == '~':
            s = nfastack.pop()
            s = ~s
            nfastack.append(s)
        elif c == '&':
            s2 = nfastack.pop()
            s1 = nfastack.pop()
            s = s1&s2
            nfastack.append(s)
        elif c == '|':
            s2 = nfastack.pop()
            s1 = nfastack.pop()
            s = s1|s2
            nfastack.append(s)
        else:
            s = NFA(c)
            nfastack.append(s)
    nfa = nfastack.pop()
    return nfa
    


# Using Graphviz to visualize the NFA

In [31]:
def VisualizeNFA(nfa):
    diag = Digraph()
    diag.attr(rankdir='LR')
    states = {j : str(i) for i,j in enumerate(nfa.states)}

    for s,i in states.items():
        if s == nfa.start :
            diag.node(name='start',label='',shape='point')
            diag.edge(tail_name='start',head_name=s,label='Start',shape='normal')
            diag.node(name=s,label=i,shape='circle',fillcolor='green',style='filled')
        elif s == nfa.final :
            diag.node(name=s,label=i,shape='doublecircle',fillcolor='red',style='filled')
        else:
            diag.node(name=s,label=i,shape='circle',fillcolor='yellow',style='filled')
    
    for t in nfa.transition:
        diag.edge(tail_name=t[0],head_name=t[1],label=t[2])

    diag.render('nfa.gv',view=True)   
    

# main()

In [99]:
regex = '(a+b)*.b'
postfix = RegexToPostfix(regex)
nfa = ConvertToNFA(postfix)
VisualizeNFA(nfa)

# Conversion of Epsilon NFA to DFA

In [33]:
def EClosure(change,closure,transitions):
    if change == False:
        return set(closure)
    else:
        prev = closure.copy()
        new = closure.copy()
        for k in prev:
            for (a,b,c) in transitions:
                if c == epsilon and a == k:
                    new.append(b)
        change = not(set(new) == set(prev))
        closure = list(set(new))
        return EClosure(change,closure,transitions)

In [34]:
class DFA:
    def __init__(self,start,final,transitions,states,alphabets):
        self.start = EClosure(True,[start],transitions)
        self.states= [EClosure(True,[s],transitions) for s in states]
        self.transition=[]
        for a in self.states:
            for c in alphabets:
                b = []
                for (i,j,k) in transitions:
                    if i in a and k == c:
                        b.append(j)
                if len(b) != 0: 
                    b = EClosure(True,b,transitions)
                    self.transition.append((a,b,c))
                    if b not in self.states:
                        self.states.append(b)
               
        self.final = []
        for x in self.states:
            if final in x:
                self.final.append(x)
        

In [87]:
class minDFA:
    def __init__(self,dfa):
        self.states = { str(j) : i for (i,j) in enumerate(dfa.states)}
        transition = [(self.states[str(s)],self.states[str(f)],c) for (s,f,c) in dfa.transition]
        dfa.final = [self.states[str(s)]  for s in dfa.final]
        self.start = self.states[str(dfa.start)]
        self.states = list(self.states.values()) 
        reachable = [self.start]
        change = True
        while change == True:
            prev = reachable
            new = reachable
            for (a,b,c) in transition:
                if a in reachable:
                    reachable.append(b)
            change = not(set(new) == set(prev))
            reachable = list(set(new))
        self.reachable = reachable
        self.transition = list()
        for (a,b,c) in transition:
            if a in reachable and b in reachable:
                    self.transition.append((a,b,c))
        self.states = self.reachable
        self.final = list()
        for i in dfa.final:
            if i in self.states:
                self.final.append(i)
                
            

In [55]:
def VisualizeDFA(dfa):
    diag = Digraph()
    diag.attr(rankdir='LR')
    states = { str(j) : str(i) for (i,j) in enumerate(dfa.states)}
    dfa.final = [str(x) for x in dfa.final]
    for s,i in states.items():
        if s == str(dfa.start) :
            diag.node(name=s,label=i,shape='circle',fillcolor='green',style='filled')
        elif s in dfa.final :
            diag.node(name=s,label=i,shape='doublecircle',fillcolor='red',style='filled')
        else:
            diag.node(name=s,label=i,shape='circle',fillcolor='yellow',style='filled')
    
    for t in dfa.transition:
        diag.edge(tail_name=str(t[0]),head_name=str(t[1]),label=t[2])

    diag.render('dfa.gv',view=True)   

In [95]:
def VisualizeMinDFA(dfa):
    diag = Digraph()
    diag.attr(rankdir='LR')
    states = { j : str(i) for (i,j) in enumerate(dfa.states)}
    for s,i in states.items():
        if s == dfa.start :
            diag.node(name=str(s),label=str(i),shape='circle',fillcolor='green',style='filled')
        elif s in dfa.final :
            diag.node(name=str(s),label=str(i),shape='doublecircle',fillcolor='red',style='filled')
        else:
            diag.node(name=str(s),label=str(i),shape='circle',fillcolor='yellow',style='filled')
    
    for t in dfa.transition:
        diag.edge(tail_name=str(t[0]),head_name=str(t[1]),label=t[2])

    diag.render('mindfa.gv',view=True)   

In [71]:
# Processing of NFA

states = {j : i for i,j in enumerate(nfa.states)}
transitions = [(states[s],states[f],c)for (s,f,c) in nfa.transition]
start = states[nfa.start]
final = states[nfa.final]
alphabets = list(set([c for (a,b,c) in transitions]))
alphabets.remove(epsilon)
states = list(set([v for v in states.values()]))

In [88]:
dfa = DFA(start,final,transitions,states,alphabets)
#VisualizeDFA(dfa)

In [89]:
mindfa = minDFA(dfa)
#VisualizeMinDFA(mindfa)

In [90]:
mindfa.reachable

[12, 4, 5]

In [91]:
mindfa.final

[12]

In [92]:
mindfa.start

4

In [93]:
mindfa.transition

[(4, 5, 'a'),
 (4, 12, 'b'),
 (5, 5, 'a'),
 (5, 12, 'b'),
 (12, 5, 'a'),
 (12, 12, 'b')]

In [96]:
VisualizeMinDFA(mindfa)